In [15]:
import sys
sys.path.append("../../../")

import os 
os.environ["WANDB_API_KEY"] = "d63df136f169bffa17486544a5339caee2aec09c"

from modules.acquire_data import DataAcquire
from modules.analyze_data import analyze_data
from modules.create_model import create_model

da = DataAcquire()

In [7]:
df_train = da.get_data_and_columns("Train")
df_test = da.get_data_and_columns("Test")

DATA ACQUIRE:COMPLETE

COLUMNS
----------------------------------------
COLUMN :  D-TYPE :  #NANS
id : int64 : 0 nans
Podcast_Name : object : 0 nans
Episode_Title : object : 0 nans
Episode_Length_minutes : float64 : 87093 nans
Genre : object : 0 nans
Host_Popularity_percentage : float64 : 0 nans
Publication_Day : object : 0 nans
Publication_Time : object : 0 nans
Guest_Popularity_percentage : float64 : 146030 nans
Number_of_Ads : float64 : 1 nans
Episode_Sentiment : object : 0 nans
Listening_Time_minutes : float64 : 0 nans

DATA ACQUIRE:COMPLETE

COLUMNS
----------------------------------------
COLUMN :  D-TYPE :  #NANS
id : int64 : 0 nans
Podcast_Name : object : 0 nans
Episode_Title : object : 0 nans
Episode_Length_minutes : float64 : 28736 nans
Genre : object : 0 nans
Host_Popularity_percentage : float64 : 0 nans
Publication_Day : object : 0 nans
Publication_Time : object : 0 nans
Guest_Popularity_percentage : float64 : 48832 nans
Number_of_Ads : float64 : 0 nans
Episode_Sentiment : 

In [8]:
df_train

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [9]:
import os
import random
import numpy as np
import pandas as pd
import wandb

from wandb.integration.lightgbm import log_summary, wandb_callback

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

/Users/takizawakeiya/Desktop/python/Takizawa-venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [46]:
df_train = da.get_data_and_columns("Train")
df_test = da.get_data_and_columns("Test")
categorical_columns = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
label_encoders = {}

for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df_train[col] = label_encoders[col].fit_transform(df_train[col].astype(str))
    df_test[col] = label_encoders[col].fit_transform(df_test[col].astype(str))

DATA ACQUIRE:COMPLETE

COLUMNS
----------------------------------------
COLUMN :  D-TYPE :  #NANS
id : int64 : 0 nans
Podcast_Name : object : 0 nans
Episode_Title : object : 0 nans
Episode_Length_minutes : float64 : 87093 nans
Genre : object : 0 nans
Host_Popularity_percentage : float64 : 0 nans
Publication_Day : object : 0 nans
Publication_Time : object : 0 nans
Guest_Popularity_percentage : float64 : 146030 nans
Number_of_Ads : float64 : 1 nans
Episode_Sentiment : object : 0 nans
Listening_Time_minutes : float64 : 0 nans

DATA ACQUIRE:COMPLETE

COLUMNS
----------------------------------------
COLUMN :  D-TYPE :  #NANS
id : int64 : 0 nans
Podcast_Name : object : 0 nans
Episode_Title : object : 0 nans
Episode_Length_minutes : float64 : 28736 nans
Genre : object : 0 nans
Host_Popularity_percentage : float64 : 0 nans
Publication_Day : object : 0 nans
Publication_Time : object : 0 nans
Guest_Popularity_percentage : float64 : 48832 nans
Number_of_Ads : float64 : 0 nans
Episode_Sentiment : 

## モデル作成で使用する関数

In [47]:
class CFG:
    exp_name = 'exp001'
    test_size = 0.2
    random_state = 529
    learning_rate = 0.1
    num_leaves = 31
    n_estimators = 10000
    feature_fraction = 0.9
    stopping_rounds = 50
    objective = 'regression'
    metric = 'rmse'
    features = ['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre', 
               'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time', 
               'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment']
    target = ['Listening_Time_minutes']
    # カテゴリカル特徴量のリストを追加
    categorical_features = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 
                          'Publication_Time', 'Episode_Sentiment']
    

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


def class_to_dict(obj):
    return {k: getattr(obj, k) for k in dir(obj) if not k.startswith('_') and not callable(getattr(obj, k))}


In [48]:
config = CFG()
seed_everything(config.random_state)
class_to_dict(config)


{'categorical_features': ['Podcast_Name',
  'Episode_Title',
  'Genre',
  'Publication_Day',
  'Publication_Time',
  'Episode_Sentiment'],
 'exp_name': 'exp001',
 'feature_fraction': 0.9,
 'features': ['Podcast_Name',
  'Episode_Title',
  'Episode_Length_minutes',
  'Genre',
  'Host_Popularity_percentage',
  'Publication_Day',
  'Publication_Time',
  'Guest_Popularity_percentage',
  'Number_of_Ads',
  'Episode_Sentiment'],
 'learning_rate': 0.1,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 31,
 'objective': 'regression',
 'random_state': 529,
 'stopping_rounds': 50,
 'target': ['Listening_Time_minutes'],
 'test_size': 0.2}

## Wandbの起動

In [49]:
wandb.login()

True

In [50]:
wandb.init(project="exp001",
           config=class_to_dict(config),
           name=config.exp_name,
           )

In [32]:
X_train, X_test, y_train, y_test = train_test_split(df_train[config.features],
                                                    df_train[config.target],
                                                    test_size=config.test_size,
                                                    random_state=config.random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'learning_rate': config.learning_rate,
    'num_leaves': config.num_leaves,
    'objective': config.objective,
    'metric': config.metric,
    'feature_fraction': config.feature_fraction,
}

model = lgb.train(
                  params, 
                  train_data,
                  num_boost_round=config.n_estimators,
                  callbacks = [
                               lgb.early_stopping(stopping_rounds=config.stopping_rounds, verbose=True),
                               lgb.log_evaluation(period=100), wandb_callback()
                              ],
                  valid_sets = [train_data, test_data],
                  )
log_summary(model, save_model_checkpoint=True)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001985 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 944
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 10
[LightGBM] [Info] Start training from score 45.410277
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 13.013	valid_1's rmse: 13.1011
[200]	training's rmse: 12.8968	valid_1's rmse: 13.0687
[300]	training's rmse: 12.801	valid_1's rmse: 13.0413
[400]	training's rmse: 12.7149	valid_1's rmse: 13.0248
[500]	training's rmse: 12.6335	valid_1's rmse: 13.0128
[600]	training's rmse: 12.564	valid_1's rmse: 13
[700]	training's rmse: 12.4895	valid_1's rmse: 12.9883
[800]	training's rmse: 12.4216	valid_1's rmse: 12.9789
[900]	training's rmse: 12.3602	valid_1's rmse: 12.9697
[1000]	training's rmse: 12.2982	valid_1's rms

In [33]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

# モデルの保存
model.save_model(f'lgb_model_{config.exp_name}.txt')
wandb.finish()

RMSE: 12.920982971752377


/Users/takizawakeiya/Desktop/python/Takizawa-venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


iteration,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇██
training_rmse,█▅▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_1_rmse,█▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,1830
iteration,1879


In [43]:
X_prediction

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral
...,...,...,...,...,...,...,...,...,...,...
249995,Mind & Body,Episode 100,21.05,Health,65.77,Saturday,Evening,96.40,3.0,Negative
249996,Joke Junction,Episode 85,85.50,Comedy,41.47,Saturday,Night,30.52,2.0,Negative
249997,Joke Junction,Episode 63,12.11,Comedy,25.92,Thursday,Evening,73.69,1.0,Neutral
249998,Market Masters,Episode 46,113.46,Business,43.47,Friday,Night,93.59,3.0,Positive


In [51]:
X_prediction = df_test[config.features]
y_prediction = model.predict(X_prediction, num_iteration=model.best_iteration)
y_prediction

array([55.77718019, 18.64613682, 50.65566287, ...,  6.26581212,
       75.25778717, 57.74948389])

In [54]:
df_submit = da.get_submit_data()
df_submit.columns = df_submit.iloc[0]
df_submit = df_submit.iloc[1:].reset_index(drop=True)

df_submit["Listening_Time_minutes"] = y_prediction
df_submit.to_csv(f'submit_{config.exp_name}.csv', index=False)

In [57]:
ls ~/.kaggle

kaggle.json


In [59]:
!kaggle competitions submit -c playground-series-s5e4 -f submit_exp001.csv -m "Initial submission"

/Users/takizawakeiya/Desktop/python/Takizawa-venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
100%|███████████████████████████████████████| 6.01M/6.01M [00:28<00:00, 218kB/s]
Successfully submitted to Predict Podcast Listening Time